In [1]:
import julia
julia.install()

[ Info: Julia version info


Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
      "Manjaro Linux"
  uname: Linux 6.5.13-7-MANJARO #1 SMP PREEMPT_DYNAMIC Wed Dec 20 07:15:58 UTC 2023 x86_64 unknown
  CPU: 13th Gen Intel(R) Core(TM) i5-13400F: 
                 speed         user         nice          sys         idle          irq
       #1-16  4100 MHz     705288 s    1595013 s     187886 s  188725410 s      25396 s
       
  Memory: 62.63386535644531 GB (1541.53515625 MB free)
  Uptime: 1.196453e6 sec
  Load Avg:  0.63  0.2  0.06
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, goldmont)
Environment:
  HOME = /home/iboero
  PATH = /home/iboero/miniconda3/envs/proy/bin:/home/iboero/miniconda3/condabin:/home/iboero/.vscode-server/bin/8b3775030ed1a69b13e4f4c628c612102e30a681/bin/remote-cli:/home/iboero/julia/bin:/home/iboero/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/opt/cuda/bin:/opt/cuda/nsight_compute:/opt/cuda/nsight_system

[ Info: Julia executable: /home/iboero/julia/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
│     libpython: /home/iboero/miniconda3/envs/proy/lib/libpython3.11.so.1.0
│ Python:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
└     libpython: 


In [2]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

In [3]:
import pandapower as pp
import pandapower.networks as nw
from copy import deepcopy
import pandapower.plotting as plot
import numpy as np
import pandas as pd
import os
import numba

## Generamos el dataset

In [ ]:
net = pp.networks.case118()
net.load['controllable'] = False
net.gen['controllable'] = True

columns_load = []
columns_gen = []
columns_ext_grid = []
columns = []

for i in range(len(net.load['p_mw'])):
  columns_load.append(f'load_{i}')
for i in range(len(net.gen['p_mw'])):
  columns_gen.append(f'gen_{i}')
for i in range(len(net.ext_grid['bus'])):
  columns_ext_grid.append(f'ext_grid_{i}')
columns = columns_load + columns_gen + columns_ext_grid

dataset_30 = pd.DataFrame(columns=columns)

load_p_nom = net.load['p_mw']
load_q_nom = net.load['q_mvar']
for j in range(10):
  dataset_30 = pd.DataFrame(columns=columns)
  for i in range(1000):
    print('iteracion:', j, '/' , i)
    uniforme = np.random.uniform(0.9,1.1,size=len(load_p_nom))
    uniforme_react = np.random.uniform(0.9,1.1,size=len(load_q_nom))
    net.load['p_mw'] = uniforme*load_p_nom
    net.load['q_mvar'] = uniforme_react*load_q_nom
    try:
      pp.runpm_ac_opf(net)
      data_i = np.concatenate([net.res_load['p_mw'] + net.res_load['q_mvar']*1j,net.res_gen['p_mw'] + net.res_gen['q_mvar']*1j,net.res_ext_grid['p_mw'] + net.res_ext_grid['q_mvar']*1j]).reshape(1,-1)
      data_i_pd = pd.DataFrame(data_i,columns=columns)
      dataset_30 = pd.concat([dataset_30,data_i_pd],ignore_index=True,axis=0)
    except:
      pass
  dataset_30.to_csv(f'./dataset_damian_118/dataset_{j}.csv',index=False)

In [ ]:
net = pp.networks.case118()

In [68]:
# net.load['controllable'] = False
# net.gen['controllable'] = True

load_p_nom = net.load['p_mw']
load_q_nom = net.load['q_mvar']

uniforme = np.random.uniform(0.9,1.1,size=len(load_p_nom))
uniforme_react = np.random.uniform(0.9,1.1,size=len(load_q_nom))

net.load['p_mw'] = uniforme*load_p_nom
net.load['q_mvar'] = uniforme_react*load_q_nom
pp.runpp(net)

X = torch.Tensor(net.res_bus.values).shape


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



## Lo transformamos en Numpy

In [4]:
datos_path_list = ['./dataset_damian_118/']

In [5]:
net = pp.networks.case118()
net.load['controllable'] = False
net.gen['controllable'] = True

columns_bus = []
for i in range(len(net.bus['name'])):
  columns_bus.append(f'bus_{i}')

/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_ax

In [4]:
net = pp.networks.case118()


NameError: name 'pp' is not defined

In [9]:
import warnings

X = []
for datos_path in datos_path_list:
  for f, path in enumerate(os.listdir(datos_path)):
    data = pd.read_csv(datos_path+path)
    for i in range(len(data)):
      q_i = data.values[i].astype(complex)[:len(net.load)].imag
      p_i = data.values[i].astype(complex)[:len(net.load)].real
      net.load['p_mw'] = p_i
      net.load['q_mvar'] = q_i
      pp.runpp(net,numba=False)
      X_i = np.array(net.res_bus.values)
      X.append(X_i)
      print(i)
X = np.array(X)
X.shape

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

(10000, 118, 4)

In [10]:
np.save("dataset_damian_np/features_118.npy",X)

In [11]:
dataset_np_final = []
for datos_path in datos_path_list:
  for f, path in enumerate(os.listdir(datos_path)):
    data = pd.read_csv(datos_path+path)
    for i in data.columns:
      data[i] = data[i].apply(complex)
    zero_data = np.zeros(shape=(data.shape[0],len(columns_bus)))
    dataset_bus = pd.DataFrame(zero_data,columns=columns_bus)
    dataset_np = []

    for i in range(len(net.bus['name'])):
      column_i = np.zeros((dataset_bus.shape[0],4))
      for idx, j in enumerate(net.gen['bus']):
        if i==j:
          column_i[:,0]= np.real(data['gen_'+str(idx)])
          column_i[:,1]= np.imag(data['gen_'+str(idx)])
      for idx, j in enumerate(net.ext_grid['bus']):
        if i==j:
          column_i[:,0]= np.real(data['ext_grid_'+str(idx)])
          column_i[:,1]= np.imag(data['ext_grid_'+str(idx)])
      for idx, j in enumerate(net.load['bus']):
        if i==j:
          column_i[:,2]= np.real(data['load_'+str(idx)])
          column_i[:,3]= np.imag(data['load_'+str(idx)])

      dataset_np.append(column_i)

    dataset_np = np.array(dataset_np).transpose(1,0,2)
    dataset_np_final.append(dataset_np)

dataset_np_final = np.concatenate(dataset_np_final)
np.save('./dataset_damian_np/dataset_118_np.npy',dataset_np_final)

NameError: name 'columns_bus' is not defined

## Entrenamos las Redes Neuronales

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import pandapower as pp
import networkx as nx

In [7]:
datos_path = './dataset_damian_np'
data = np.load(datos_path+'/dataset_118_np.npy')
X = np.load(datos_path+'/features_118.npy')
net = pp.networks.case118()
data.shape


/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_ax

(10000, 118, 4)

In [8]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
device

'cuda'

In [13]:
z_lineas =  net.line[['r_ohm_per_km', 'x_ohm_per_km', 'c_nf_per_km']].to_numpy() * 100 / np.expand_dims((net.bus['vn_kv'].to_numpy()[net.line['from_bus'].to_numpy()])**2,axis=1)
z_lineas

array([[3.03000000e-02, 9.99000000e-02, 1.85774565e+00],
       [1.29000000e-02, 4.24000000e-02, 7.91370390e-01],
       [1.76000000e-03, 7.98000000e-03, 1.53593144e-01],
       [2.41000000e-02, 1.08000000e-01, 2.07716442e+00],
       [1.19000000e-02, 5.40000000e-02, 1.04297059e+00],
       [4.59000000e-03, 2.08000000e-02, 4.02267758e-01],
       [2.44000000e-03, 3.05000000e-02, 2.17569808e+00],
       [2.58000000e-03, 3.22000000e-02, 2.30301949e+00],
       [2.09000000e-02, 6.88000000e-02, 1.27848007e+00],
       [2.03000000e-02, 6.82000000e-02, 1.27116612e+00],
       [5.95000000e-03, 1.96000000e-02, 3.67160754e-01],
       [1.87000000e-02, 6.16000000e-02, 1.14975439e+00],
       [4.84000000e-02, 1.60000000e-01, 2.96946745e+00],
       [8.62000000e-03, 3.40000000e-02, 6.39240037e-01],
       [2.22500000e-02, 7.31000000e-02, 1.37209875e+00],
       [2.15000000e-02, 7.07000000e-02, 1.32821500e+00],
       [7.44000000e-02, 2.44400000e-01, 4.58438965e+00],
       [5.95000000e-02, 1.95000

In [9]:
lineas = net.line[['from_bus', 'to_bus']].to_numpy().astype(np.int32)
trafos = net.trafo[['hv_bus', 'lv_bus']].to_numpy().astype(np.int32)
edge_index = np.append(lineas, trafos,axis=0)
edge_index = torch.Tensor(edge_index).t().type(torch.int64).to(device)
edge_index

tensor([[  0,   0,   3,   2,   4,   5,   7,   8,   3,   4,  10,   1,   2,   6,
          10,  11,  12,  13,  11,  14,  15,  16,  17,  18,  14,  19,  20,  21,
          22,  22,  24,  26,  27,   7,  25,  16,  28,  22,  30,  26,  14,  18,
          34,  34,  32,  33,  33,  36,  36,  29,  38,  39,  39,  40,  42,  33,
          43,  44,  45,  45,  46,  41,  41,  44,  47,  48,  48,  50,  51,  52,
          48,  48,  53,  53,  54,  55,  49,  55,  50,  53,  55,  55,  54,  58,
          58,  59,  59,  60,  62,  37,  63,  48,  48,  61,  61,  65,  46,  48,
          68,  23,  69,  23,  70,  70,  69,  69,  68,  73,  75,  68,  74,  76,
          77,  76,  76,  78,  76,  81,  82,  82,  83,  84,  84,  84,  87,  88,
          88,  89,  88,  88,  90,  91,  91,  92,  93,  79,  81,  93,  79,  79,
          79,  91,  93,  94,  95,  97,  98,  99,  91, 100,  99,  99, 102, 102,
          99, 103, 104, 104, 104, 105, 107, 102, 108, 109, 109,  16,  31,  31,
          26, 113,  11,  74,  75,   7,  25,  29,  37

In [10]:
idx_gen = net.gen["bus"].to_numpy()
idx_grid = net.ext_grid["bus"].to_numpy()
idx_gens = np.append(idx_gen,idx_grid,axis=0)

feature_mask = np.zeros(len(net.bus.index), dtype=int)
feature_mask[idx_gens] = 1
feature_mask = torch.Tensor(feature_mask).type(torch.int32).to(device)

In [11]:
y_tensor = torch.Tensor(data[:,:,0]).to(device)
X_tensor = torch.Tensor(X).to(device)
y_tensor_global = y_tensor[:, feature_mask == 1]

In [24]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import ReLU, LogSoftmax, BatchNorm1d, LeakyReLU
from torch_geometric.nn import GCNConv, TAGConv

class GNNWithFeatureMask(nn.Module):
    def __init__(self, dim, edge_index, num_layers, K, feature_mask,batchnorm=False):
        super(GNNWithFeatureMask, self).__init__()
        self.feature_mask = feature_mask
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        self.batchnorm = torch.nn.ModuleList()

        for layer in range(num_layers):
          self.convs.append(TAGConv(dim[layer],dim[layer+1],K[layer],bias=True))
          self.batchnorm.append(BatchNorm1d(118))

        self.edge_index = edge_index
        self.relu = LeakyReLU()

    def forward(self, x):
        # Separate the feature mask from the node features
        num_features = x.size(2)
        feature_mask = self.feature_mask  # Extract the feature mask

        # Apply the GNN to the node features
        num_layers = self.num_layers
        convs = self.convs
        batchnorm = self.batchnorm
        relu = self.relu

        out = x
        for layer in range(num_layers-1):
          out = convs[layer](out,edge_index)
          if batchnorm:
            out = batchnorm[layer](out)
          out = relu(out)

        out = convs[-1](out,edge_index)
        x = out.squeeze()

        # Apply the feature mask to the output features
        x = x * feature_mask

        return x, feature_mask
    



class GNN_local(nn.Module):
    def __init__(self, dim, edge_index, num_layers, K, feature_mask,batchnorm=False):
        super(GNNWithFeatureMask, self).__init__()
        self.feature_mask = feature_mask
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        self.batchnorm = torch.nn.ModuleList()

        for layer in range(num_layers):
          self.convs.append(TAGConv(dim[layer],dim[layer+1],K[layer],bias=True))
          self.batchnorm.append(BatchNorm1d(118))

        self.edge_index = edge_index
        self.relu = LeakyReLU()

    def forward(self, x):
        # Separate the feature mask from the node features
        num_features = x.size(2)
        feature_mask = self.feature_mask  # Extract the feature mask

        # Apply the GNN to the node features
        num_layers = self.num_layers
        convs = self.convs
        batchnorm = self.batchnorm
        relu = self.relu

        out = x
        for layer in range(num_layers-1):
          out = convs[layer](out,edge_index)
          if batchnorm:
            out = batchnorm[layer](out)
          out = relu(out)

        out = convs[-1](out,edge_index)
        x = out.squeeze()

        # Apply the feature mask to the output features
        x = x * feature_mask

        return x, feature_mask


class GNN_global(nn.Module):
    def __init__(self, dim, edge_index, num_layers, K, num_nodes, num_gens):
        super(GNN_global, self).__init__()

        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        self.num_nodes = num_nodes
        self.num_gens = num_gens
        self.dim = dim
        
        for layer in range(num_layers-1):
          self.convs.append(TAGConv(dim[layer],dim[layer+1],K[layer],bias=True))
        
        self.fcnn = torch.nn.Linear(dim[-2]*num_nodes,num_gens)

        self.edge_index = edge_index
        self.relu = LeakyReLU()

    def forward(self, x):

        # Apply the GNN to the node features
        num_layers = self.num_layers
        convs = self.convs
        relu = self.relu
        fcnn = self.fcnn
        out = x
        for layer in range(num_layers-1):
          out = convs[layer](out,edge_index)
          out = relu(out)
        out = out.reshape(-1,self.dim[-2]*self.num_nodes)
        out = fcnn(out)
        x = out.squeeze()

        return x


In [32]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

dataset = TensorDataset(X_tensor, y_tensor)
data_train,data_test = train_test_split(dataset,test_size=0.2,random_state=42)
data_train,data_val = train_test_split(data_train,test_size=0.1,random_state=42)
# Create data loaders for training, validation, and test sets
batch_size = 128
train_loader = DataLoader(data_train, batch_size=batch_size)
val_loader = DataLoader(data_val, batch_size=batch_size)
test_loader = DataLoader(data_test, batch_size=batch_size)



In [47]:
dataset_global = TensorDataset(X_tensor, y_tensor_global)
data_train_global ,data_test_global  = train_test_split(dataset_global ,test_size=0.2,random_state=42)
data_train_global ,data_val_global  = train_test_split(data_train_global ,test_size=0.1,random_state=42)
# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader_global  = DataLoader(data_train_global , batch_size=batch_size)
val_loader_global  = DataLoader(data_val_global , batch_size=batch_size)
test_loader_global  = DataLoader(data_test_global , batch_size=batch_size)

In [40]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for data in train_loader:
        optimizer.zero_grad()
        output, feature_mask = model(data[0])
        target = data[1]

        # Compute the MSE loss only for nodes with missing features
        target = torch.masked_select(target, feature_mask.unsqueeze(0).bool()).view(target.shape[0],torch.sum(feature_mask))
        output = torch.masked_select(output, feature_mask.unsqueeze(0).bool()).view(output.shape[0],torch.sum(feature_mask))
        loss = criterion(output, target)  # target should contain true values for nodes with missing features
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate(model, data_loader, criterion,test=False):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for data in data_loader:
            output, feature_mask = model(data[0])
            target = data[1]

            # Compute the MSE loss only for nodes with missing features
            target = torch.masked_select(target, feature_mask.unsqueeze(0).bool()).view(target.shape[0],torch.sum(feature_mask))
            output = torch.masked_select(output, feature_mask.unsqueeze(0).bool()).view(output.shape[0],torch.sum(feature_mask))
            loss = criterion(output, target)  # target should contain true values for nodes with missing features
            total_loss += loss.item()
            if test:
              print(output)
              print(target)


    return total_loss / len(data_loader)

In [56]:
def train_global(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    for data in train_loader:
        optimizer.zero_grad()
        output = model(data[0])
        target = data[1]
        loss = criterion(output, target) 
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Calcular la métrica
        metric = normalized_error(output, target)
        total_metric += metric.item()

    avg_loss = total_loss / len(train_loader)
    avg_metric = total_metric / len(train_loader)

    return avg_loss, avg_metric


def evaluate_global(model, data_loader, criterion,test=False):
    model.eval()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    with torch.no_grad():
        for data in data_loader:
            output = model(data[0])
            target = data[1]
            loss = criterion(output, target)  # target should contain true values for nodes with missing features
            total_loss += loss.item()
            # Calcular la métrica
            metric = normalized_error(output, target)
            total_metric += metric.item()

            if test:
              print(output)
              print(target)

    avg_loss = total_loss / len(data_loader)
    avg_metric = total_metric / len(data_loader)
    return avg_loss, avg_metric

In [58]:
dim = [4,128,64,1]
num_layers = 3
K = [4,4]


model_global = GNN_global(dim, edge_index, num_layers, K, 118, 54).to(device)
optimizer = torch.optim.Adam(model_global.parameters(), lr=0.0001)
criterion = nn.MSELoss()  # Change the loss function as needed

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    train_loss, train_metric = train_global(model_global, train_loader_global, optimizer, criterion)
    val_loss, val_metric = evaluate_global(model_global, val_loader_global, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f},  Val Loss: {val_loss:.4f}, Val Metric: {val_metric:.4f}")

# Evaluate the model on the test set
test_loss, test_metric = evaluate_global(model_global, test_loader_global, criterion)
print(f"Test Loss: {test_loss:.5f}, Test Metric: {test_metric:.5f}")
train_loss, train_metric = evaluate_global(model_global, train_loader_global, criterion)
print(f"Train Loss: {train_loss:.5f}, Train Metric: {train_metric:.5f}")

Epoch 1/100, Train Loss: 2580.0907, Train Metric: 0.1789,  Val Loss: 2.8115, Val Metric: 0.0107
Epoch 2/100, Train Loss: 2.6629, Train Metric: 0.0104,  Val Loss: 2.4532, Val Metric: 0.0100
Epoch 3/100, Train Loss: 2.3724, Train Metric: 0.0098,  Val Loss: 2.1989, Val Metric: 0.0095
Epoch 4/100, Train Loss: 2.1236, Train Metric: 0.0093,  Val Loss: 1.9474, Val Metric: 0.0089
Epoch 5/100, Train Loss: 1.9197, Train Metric: 0.0088,  Val Loss: 1.7409, Val Metric: 0.0084
Epoch 6/100, Train Loss: 1.7454, Train Metric: 0.0084,  Val Loss: 1.6055, Val Metric: 0.0081
Epoch 7/100, Train Loss: 1.5925, Train Metric: 0.0081,  Val Loss: 1.4816, Val Metric: 0.0078
Epoch 8/100, Train Loss: 1.4576, Train Metric: 0.0077,  Val Loss: 1.4002, Val Metric: 0.0076
Epoch 9/100, Train Loss: 1.3444, Train Metric: 0.0074,  Val Loss: 1.2829, Val Metric: 0.0072
Epoch 10/100, Train Loss: 1.2525, Train Metric: 0.0071,  Val Loss: 1.1629, Val Metric: 0.0069
Epoch 11/100, Train Loss: 1.1707, Train Metric: 0.0069,  Val Loss:

In [50]:
import torch
import torch.nn as nn

class NormalizedError(nn.Module):
    def __init__(self):
        super(NormalizedError, self).__init__()

    def forward(self, y_pred, y_true):
        """
        Calcula el error normalizado entre la predicción y el valor real.

        Parámetros:
        y_pred (Tensor): Predicciones del modelo.
        y_true (Tensor): Valores reales.

        Retorna:
        Tensor: Error normalizado.
        """
        # Asegurarse de que las predicciones y los valores reales están en la CPU
        y_pred = y_pred.cpu()
        y_true = y_true.cpu()

        # Calcular la norma de la diferencia entre la predicción y el valor real
        numerator = torch.norm(y_pred - y_true)

        # Calcular la norma del valor real
        denominator = torch.norm(y_true)

        # Evitar la división por cero
        if denominator == 0:
            return torch.tensor(0.0)

        # Calcular y retornar el error normalizado
        return numerator / denominator


In [49]:
X_val_0, y_val_0 = next(iter(val_loader_global))
y_val_0_pred  = model_global(X_val_0)

# # res = torch.vstack()
# print(y_val_0)
# print(y_val_0_pred)

torch.sqrt(     torch.norm(y_val_0_pred.cpu()-y_val_0.cpu()) / torch.norm(y_val_0.cpu())   )

tensor(0.0553, grad_fn=<SqrtBackward0>)

In [ ]:
test_loss = evaluate(model, test_loader, criterion,test=True)
print(f"Test Loss: {test_loss:.4f}")